In [1]:
!pip3 install psycopg2

In [2]:
import psycopg2
import numpy as np

In [3]:
# establishing the connection
conn = psycopg2.connect(
    database="postgres",
    user='postgres',
    password='postgrespassword',
    host='localhost',
    port='5432'
)

In [4]:
# Creating a cursor object using the cursor() method
cursor = conn.cursor()

# Check if the volunteer_table_test table exists
cursor.execute("SELECT EXISTS (SELECT FROM information_schema.tables WHERE table_name = 'volunteer_table_test')")
table_exists = cursor.fetchone()[0]

# If the table exists, drop it
if table_exists:
    cursor.execute("DROP TABLE IF EXISTS volunteer_table_test")
    

    
    
# Creating a second cursor object using the cursor() method
cursor2 = conn.cursor()

# Check if the event_list table exists
cursor2.execute("SELECT EXISTS (SELECT FROM information_schema.tables WHERE table_name = 'event_list')")
table_exists = cursor2.fetchone()[0]

# If the table exists, drop it
if table_exists:
    cursor2.execute("DROP TABLE IF EXISTS event_list")

In [5]:
# SQL statement to create the table
sql = '''CREATE TABLE VOLUNTEER_TABLE_TEST(
   EMAIL VARCHAR(255) NOT NULL PRIMARY KEY,
   PASSWORD VARCHAR(100) NOT NULL  
)'''
cursor.execute(sql)





# SQL statement to create the table
sql2 = '''CREATE TABLE EVENT_LIST(
   EVENT_NAME VARCHAR(255) NOT NULL PRIMARY KEY,
   DATE VARCHAR(100) NOT NULL,
   TIME VARCHAR(100) NOT NULL,
   EVENT_DESCRIPTION VARCHAR(1000),
   VALID_DATE VARCHAR(1) NOT NULL
)'''
cursor2.execute(sql2)

In [6]:
# Inserting values into the credentials table
insert_stmt1 = "INSERT INTO VOLUNTEER_TABLE_TEST (EMAIL, PASSWORD) VALUES (%s, %s)"
data1 = [('TEST1', 'pw1'),
        ('TEST2', 'pw2'),
        ]
cursor.executemany(insert_stmt1, data1)

In [7]:
# Inserting values into the events table
insert_stmt2 = "INSERT INTO EVENT_LIST (EVENT_NAME, DATE, TIME, EVENT_DESCRIPTION, VALID_DATE) VALUES (%s, %s, %s, %s, %s)"
data2 = [('EVENT1', '30 April 2024', '1 pm', '',1),
        ('EVENT2', '2 May 2024', '10 am', "",1),
        ]
cursor2.executemany(insert_stmt2, data2)

In [8]:
# Display whole table
# cursor.execute("SELECT * FROM VOLUNTEER_TABLE_TEST")
# data = cursor.fetchall()[0]
# print(type(data),'\n\n', data,'\n\n', data[0])

In [9]:
cursor.execute("SELECT COUNT(EMAIL) FROM VOLUNTEER_TABLE_TEST;")
cursor2.execute("SELECT DATE, VALID_DATE from EVENT_LIST;")
# SELECT COUNT(column_name) FROM table_name;

In [10]:
volunteer_count = cursor.fetchall()[0][0]
event_data = cursor2.fetchall()

dates = [row[0] for row in event_data]
valid_dates = [row[1] for row in event_data]
# print(volunteer_count[0][0])
# print(data,'\n\n', volunteer_count,'\n\n')

In [11]:
dates, valid_dates

(['30 April 2024', '2 May 2024'], ['1', '1'])

In [12]:
print(f"Count signed up volunteer count is {volunteer_count}")
print(f"\n\nThe date and valid_date are {dates, valid_dates}")


Count signed up volunteer count is 2


The date and valid_date are (['30 April 2024', '2 May 2024'], ['1', '1'])


In [13]:
from datetime import datetime

# Check if the event date is in the future
current_date = datetime.now().date()

for event_date in dates:
    event_date = datetime.strptime(event_date, "%d %B %Y").date()
    if event_date >= current_date:
        # If the event date is in the future, set VALID_DATE to 1
        cursor2.execute("UPDATE EVENT_LIST SET VALID_DATE = 1 WHERE TO_DATE(DATE, 'DD Month YYYY') = %s", (event_date,))
    else:
        # If the event date is in the past, set VALID_DATE to 0
        cursor2.execute("UPDATE EVENT_LIST SET VALID_DATE = 0 WHERE TO_DATE(DATE, 'DD Month YYYY') = %s", (event_date,))


In [14]:
# Commit your changes in the database
conn.commit()

# Closing the connection
conn.close()